In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
data_path = os.path.join(os.path.abspath(os.getcwd()), "..", "data")

df = pd.read_csv(os.path.join(data_path, "census.csv"))

df

age          workclass   fnlgt    education   education-num  \
0       39          State-gov   77516    Bachelors              13   
1       50   Self-emp-not-inc   83311    Bachelors              13   
2       38            Private  215646      HS-grad               9   
3       53            Private  234721         11th               7   
4       28            Private  338409    Bachelors              13   
...    ...                ...     ...          ...             ...   
32556   27            Private  257302   Assoc-acdm              12   
32557   40            Private  154374      HS-grad               9   
32558   58            Private  151910      HS-grad               9   
32559   22            Private  201490      HS-grad               9   
32560   52       Self-emp-inc  287927      HS-grad               9   

            marital-status          occupation    relationship    race  \
0            Never-married        Adm-clerical   Not-in-family   White   
1       Married-civ-spouse     Exec-managerial         Husband   White   
2                 Divorced   Handlers-cleaners   Not-in-family   White   
3       Married-civ-spouse   Handlers-cleaners         Husband   Black   
4       Married-civ-spouse      Prof-specialty            Wife   Black   
...                    ...                 ...             ...     ...   
32556   Married-civ-spouse        Tech-support            Wife   White   
32557   Married-civ-spouse   Machine-op-inspct         Husband   White   
32558              Widowed        Adm-clerical       Unmarried   White   
32559        Never-married        Adm-clerical       Own-child   White   
32560   Married-civ-spouse     Exec-managerial            Wife   White   

           sex   capital-gain   capital-loss   hours-per-week  native-country  \
0         Male           2174              0               40   United-States   
1         Male              0              0               13   United-States   
2         Male              0              0               40   United-States   
3         Male              0              0               40   United-States   
4       Female              0              0               40            Cuba   
...        ...            ...            ...              ...             ...   
32556   Female              0              0               38   United-States   
32557     Male              0              0               40   United-States   
32558   Female              0              0               40   United-States   
32559     Male              0              0               20   United-States   
32560   Female          15024              0               40   United-States   

       salary  
0       <=50K  
1       <=50K  
2       <=50K  
3       <=50K  
4       <=50K  
...       ...  
32556   <=50K  
32557    >50K  
32558   <=50K  
32559   <=50K  
32560    >50K  

[32561 rows x 15 columns]

In [3]:
df.dtypes

age                 int64
 workclass         object
 fnlgt              int64
 education         object
 education-num      int64
 marital-status    object
 occupation        object
 relationship      object
 race              object
 sex               object
 capital-gain       int64
 capital-loss       int64
 hours-per-week     int64
 native-country    object
 salary            object
dtype: object

## Column names and data contain white spaces

In [4]:
df.columns

Index(['age', ' workclass', ' fnlgt', ' education', ' education-num',
       ' marital-status', ' occupation', ' relationship', ' race', ' sex',
       ' capital-gain', ' capital-loss', ' hours-per-week', ' native-country',
       ' salary'],
      dtype='object')

**df['workclass'] raises a KeyError**

## Remove spaces from column names

In [6]:
df.columns = [col.strip() for col in df.columns]

df.columns

Index(['age', 'workclass', 'fnlgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'salary'],
      dtype='object')

## Remove space from categorical columns

In [7]:
cat_features = df.select_dtypes(['object']).columns

    
df[cat_features] = df[cat_features].apply(lambda x: x.str.strip())

In [8]:
df['salary'] == '<=50K'

0         True
1         True
2         True
3         True
4         True
         ...  
32556     True
32557    False
32558     True
32559     True
32560    False
Name: salary, Length: 32561, dtype: bool

In [9]:
df['workclass']

0               State-gov
1        Self-emp-not-inc
2                 Private
3                 Private
4                 Private
               ...       
32556             Private
32557             Private
32558             Private
32559             Private
32560        Self-emp-inc
Name: workclass, Length: 32561, dtype: object

## Save the clean data

In [10]:
data_path = os.path.join(os.path.abspath(os.getcwd()), "..", "data")

df.to_csv(os.path.join(data_path, "clean_census.csv"), index=False)

## Slice Performance

In [11]:
def compute_slice_metrics(
    features,
    labels,
    predictions,
    cat_features):
    
    labels = pd.Series(np.squeeze(labels))
    predictions = pd.Series(np.squeeze(predictions))
    
    # Construct the full dataframe containing labels and predictions   
    df = pd.concat([features, labels, predictions], axis=1)
    df.columns = list(features.columns) + ['labels', 'predictions']
    
    TP = df[df['labels'] == 1].groupby(cat_features)['predictions'].sum()
    FP = df[df['labels'] == 1].groupby(cat_features)['predictions'].apply(lambda x: x.count() - x.sum())
    TN = df[df['labels'] == 0].groupby(cat_features)['predictions'].apply(lambda x: x.count() - x.sum())
    FN = df[df['labels'] == 0].groupby(cat_features)['predictions'].sum()
    
    precision = (TP / (TP + FP))
    recall = (TP / (TP + FN))
    TNR = (TN / (TN + FP))
    NPV = (TN / (TN + FN))
    
    final_df = pd.concat([precision, recall, TNR, NPV], axis=1)
    final_df.columns = ['Precision', 'Recall', 'TNR', 'NPV']
    
    return final_df

In [12]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()

labels = df['salary']
labels = lb.fit_transform(labels)

features = df.copy().drop(['salary'], axis=1)
cat_features = ['workclass']

slice_performance = compute_slice_metrics(
                        features,
                        labels,
                        labels,
                        cat_features)
        
print(slice_performance)

                  Precision  Recall  TNR  NPV
workclass                                    
?                       1.0     1.0  1.0  1.0
Federal-gov             1.0     1.0  1.0  1.0
Local-gov               1.0     1.0  1.0  1.0
Private                 1.0     1.0  1.0  1.0
Self-emp-inc            1.0     1.0  1.0  1.0
Self-emp-not-inc        1.0     1.0  1.0  1.0
State-gov               1.0     1.0  1.0  1.0
Never-worked            NaN     NaN  NaN  1.0
Without-pay             NaN     NaN  NaN  1.0
